In [1]:
import tensorflow as tf
import numpy as np
import datetime
import time

import sklearn.metrics

In [62]:
def genObs(length):
    return np.random.choice(range(2), size=length)

def genTarget(x):
    return np.sum(x)

def genSample(num, length=20):
    x = [genObs(length=length) for _ in range(num)]
    y = [genTarget(t) for t in x]
    return np.array(x), np.array(y)

def randomBatch(tensorTuple, batchSize=64):
    ids = np.random.choice(range(tensorTuple[0].shape[0]), batchSize)
    return (x[ids,] for x in tensorTuple)

In [63]:
train_x, train_y = genSample(10000)
valid_x, valid_y = genSample(1000)
valid_x[:10], valid_y[:10]

(array([[1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1],
        [0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0],
        [1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1],
        [1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0],
        [0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1],
        [0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
        [0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0],
        [0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0]]),
 array([14, 12, 11,  9, 10, 10, 17, 12, 11, 10]))

In [66]:
SEQ_LEN = 20
RNN_SIZE = [3]

InnerCell = lambda n: tf.nn.rnn_cell.BasicRNNCell(num_units=n, activation=tf.nn.relu)
#InnerCell = lambda n: tf.nn.rnn_cell.LSTMCell(num_units=n, activation=tf.nn.relu, use_peepholes=True)

tf.reset_default_graph()

tfi_x = tf.placeholder(shape=(None, SEQ_LEN), dtype=tf.float32)
tfi_y = tf.placeholder(shape=(None), dtype=tf.int64)

tfX = tf.reshape(tfi_x, shape=(tf.shape(tfi_x)[0], tf.shape(tfi_x)[1], 1))
tfYC = tf.reshape(tfi_y, shape=(tf.shape(tfi_y)[0], 1))
tfY = tf.one_hot(tfi_y, SEQ_LEN + 1, dtype=tf.float32)
tfY1 = tf.one_hot(tfi_y, 2, dtype=tf.float32)


rnnCell = tf.nn.rnn_cell.MultiRNNCell([InnerCell(s) for s in RNN_SIZE])
#rnnCell = tf.nn.rnn_cell.GRUCell(RNN_SIZE[0])

_, tfO = tf.nn.dynamic_rnn(rnnCell, inputs=tfX, dtype=tf.float32)

tfOut = tf.layers.dense(tfO[-1], 1)
#tfOut = tf.layers.dense(tfi_x, 1)

#tfLoss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tfY1, logits=tfOut))
#tfLoss = tf.reduce_mean(tf.nn.l2_loss(tf.cast(tfi_y, dtype=tf.float32) - tfOut))
#tfLoss = tf.reduce_mean(tf.abs(tf.cast(tfi_y, dtype=tf.float32) - tfOut))
tfLoss = tf.reduce_mean(tf.pow(tf.cast(tfYC, dtype=tf.float32) - tfOut, 2))
tfTrain = tf.train.AdamOptimizer(1e-2).minimize(tfLoss)

#tfOutP = tf.nn.softmax(tfOut)
tfOutR = tf.cast(tf.round(tfOut),dtype=tf.int64)

tfAccuracy = tf.reduce_mean(tf.cast(tf.equal(tfYC, tf.cast(tf.round(tfOut),dtype=tf.int64)), dtype=tf.float32))
tffw = tf.summary.FileWriter('D:/Jupyter/Logs/00_A', tf.get_default_graph())
print('Graph creation complete')

Graph creation complete


In [68]:
batch_size = 1000
num_steps  = 50
num_epochs = 20

valid_batch = {tfi_x: valid_x, tfi_y: valid_y}
with tf.Session() as tfs:
    tfs.run(tf.global_variables_initializer())
    for i in range(num_epochs):
        mini_x, mini_y = randomBatch((train_x, train_y), batchSize=batch_size)
        train_batch = {tfi_x:mini_x, tfi_y:mini_y}
        l0 = tfLoss.eval(feed_dict=train_batch)
        t0 = time.perf_counter()
        for j in range(num_steps):
            tfTrain.run(feed_dict=train_batch)
        t1 = time.perf_counter()
        l1 = tfLoss.eval(feed_dict=train_batch)
        lv = np.sqrt(tfLoss.eval(feed_dict=valid_batch))
        ac = tfAccuracy.eval(feed_dict=valid_batch)
        
        print('Epoch {0} ({1:1.3} sec): loss changed from {2:1.3} to {3:1.3}\t\tVL:{4:1.3f}\t\tAC:{5:1.3f}'.format(i,t1-t0,l0,l1,lv,ac))
    valid_r = tfs.run(tfOutR, feed_dict=valid_batch)

Epoch 0 (0.255 sec): loss changed from 1.08e+02 to 91.6		VL:9.584		AC:0.000
Epoch 1 (0.251 sec): loss changed from 90.6 to 6.53		VL:2.601		AC:0.145
Epoch 2 (0.254 sec): loss changed from 6.52 to 4.68		VL:2.206		AC:0.164
Epoch 3 (0.258 sec): loss changed from 4.73 to 4.26		VL:2.101		AC:0.168
Epoch 4 (0.219 sec): loss changed from 4.14 to 3.88		VL:2.034		AC:0.175
Epoch 5 (0.224 sec): loss changed from 4.04 to 3.77		VL:1.972		AC:0.189
Epoch 6 (0.206 sec): loss changed from 3.84 to 3.67		VL:1.924		AC:0.205
Epoch 7 (0.213 sec): loss changed from 3.52 to 3.34		VL:1.877		AC:0.204
Epoch 8 (0.212 sec): loss changed from 3.35 to 3.19		VL:1.835		AC:0.210
Epoch 9 (0.208 sec): loss changed from 3.19 to 3.02		VL:1.790		AC:0.213
Epoch 10 (0.21 sec): loss changed from 2.98 to 2.82		VL:1.740		AC:0.224
Epoch 11 (0.207 sec): loss changed from 3.03 to 2.78		VL:1.676		AC:0.226
Epoch 12 (0.213 sec): loss changed from 2.58 to 2.33		VL:1.593		AC:0.238
Epoch 13 (0.215 sec): loss changed from 2.4 to 2.06		VL:1.

In [69]:
valid_y[:15],valid_r[:15]

(array([14, 12, 11,  9, 10, 10, 17, 12, 11, 10,  6, 13, 10,  9, 12]),
 array([[14],
        [12],
        [11],
        [ 9],
        [10],
        [10],
        [17],
        [12],
        [11],
        [10],
        [ 6],
        [13],
        [10],
        [ 9],
        [12]], dtype=int64))